# Bring Your Own Model with SageMaker Script Mode

### Overview

This notebook will demonstrate how you can bring your own model by using custom training and inference scripts, similar to those you would use outside of SageMaker, with SageMaker's prebuilt containers for various frameworks like Scikit-learn, PyTorch, and XGBoost.

SageMaker Script Mode is flexible so you'll also be seeing examples of how to include your own dependencies, such as a custom Python library, in your training and inference.

The following diagram provides a solution overview:

<img title="SageMaker Script Mode" alt="Solution diagram" src="solution-diagram.jpg">

### Prerequisites

To follow along, you need to create an IAM role, SageMaker Notebook instance, and S3 bucket. You may click on the CloudFormation button which will create the aforementioned resources and clone the `amazon-sagemaker-examples` GitHub repo into the notebook instance. [![Launch Stack](https://s3.amazonaws.com/cloudformation-examples/cloudformation-launch-stack.png)](https://console.aws.amazon.com/cloudformation/home#/stacks/new?stackName=ScriptModeDemo&templateURL=https://script-mode-blog.s3.amazonaws.com/script-mode-blog-cfn.yml). Give the S3bucket a unique name; you can also give the CloudFormation stack and notebook unique names such as "script mode". You can leave the other default settings in the CloudFormation template.

Once the SageMaker Notebook instance is created, choose `conda_python3` as the kernel.

### Imports

In [2]:
pip install -U sagemaker sklearn

Note: you may need to restart the kernel to use updated packages.


In [3]:
import sagemaker
import subprocess
import sys
import random
import math
import pandas as pd
import os
import boto3
import numpy as np
from sklearn.preprocessing import StandardScaler
from sagemaker.pytorch import PyTorch
from sagemaker.xgboost import XGBoost
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.serializers import NumpySerializer, JSONSerializer, CSVSerializer
from sagemaker.deserializers import NumpyDeserializer, JSONDeserializer
from sagemaker.predictor import Predictor
from generate_synthetic_housing_data import *

Make sure your SageMaker version is updated.

In [4]:
# SageMaker Python SDK version 2.x is required
original_version = sagemaker.__version__
if sagemaker.__version__ != "2.24.1":
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sagemaker==2.24.1"])
    import importlib

    importlib.reload(sagemaker)

### Parameters

In [5]:
random.seed(42)

# Useful SageMaker variables
try:
    # You're using a SageMaker notebook
    sess = sagemaker.Session()
    bucket = sess.default_bucket()
    role = sagemaker.get_execution_role()
except ValueError:
    # You're using a notebook somewhere else
    print("Setting role and SageMaker session manually...")
    bucket = "bobby-demo"
    region = "us-west-2"

    iam = boto3.client("iam")
    sagemaker_client = boto3.client("sagemaker")

    sagemaker_execution_role_name = (
        "AmazonSageMaker-ExecutionRole-20200630T141851"  # Change this to your role name
    )
    role = iam.get_role(RoleName=sagemaker_execution_role_name)["Role"]["Arn"]
    boto3.setup_default_session(region_name=region, profile_name="default")
    sess = sagemaker.Session(sagemaker_client=sagemaker_client, default_bucket=bucket)

# Local data paths
train_dir = os.path.join(os.getcwd(), "data/train")
test_dir = os.path.join(os.getcwd(), "data/test")
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Data paths in S3
s3_prefix = "script-mode-workflow"
csv_s3_prefix = f"{s3_prefix}/csv"
csv_s3_uri = f"s3://{bucket}/{s3_prefix}/csv"
numpy_train_s3_prefix = f"{s3_prefix}/numpy/train"
numpy_train_s3_uri = f"s3://{bucket}/{numpy_train_s3_prefix}"
numpy_test_s3_prefix = f"{s3_prefix}/numpy/test"
numpy_test_s3_uri = f"s3://{bucket}/{numpy_test_s3_prefix}"
csv_train_s3_uri = f"{csv_s3_uri}/train"
csv_test_s3_uri = f"{csv_s3_uri}/test"

# Enable Local Mode training
enable_local_mode_training = False

# Endpoint names
sklearn_endpoint_name = "randomforestregressor-endpoint"
pytorch_endpoint_name = "pytorch-endpoint"
xgboost_endpoint_name = "xgboost-endpoint"

In [6]:
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/local_mode_setup.sh
!wget -q https://raw.githubusercontent.com/aws-samples/amazon-sagemaker-script-mode/master/daemon.json
!/bin/bash ./local_mode_setup.sh

./local_mode_setup.sh: line 45: docker: command not found
./local_mode_setup.sh: line 47: docker: command not found
./local_mode_setup.sh: line 51: sudo: command not found
./local_mode_setup.sh: line 54: docker: command not found
./local_mode_setup.sh: line 55: ip: command not found
./local_mode_setup.sh: line 56: ip: command not found
./local_mode_setup.sh: line 59: sudo: command not found
./local_mode_setup.sh: line 60: sudo: command not found


### Prepare Synthetic Housing Data

For all the examples below, we'll be generating a synthetic housing dataset.

In [7]:
df = generate_houses(1506)

# Get training columns
train_cols = list(df.columns)
del train_cols[-1]
train_cols

# Split data
training_index = math.floor(0.8 * df.shape[0])
x_train, y_train = df[train_cols][:training_index], df.PRICE[:training_index]
x_test, y_test = df[train_cols][training_index:], df.PRICE[training_index:]

# Scale price
y_train = y_train / 100000
y_test = y_test / 100000

# Standardize data
x_train_np = StandardScaler().fit_transform(x_train)
x_test_np = StandardScaler().fit_transform(x_test)

In [8]:
x_train.head()

,YEAR_BUILT,SQUARE_FEET,NUM_BEDROOMS,NUM_BATHROOMS,LOT_ACRES,GARAGE_SPACES,FRONT_PORCH,DECK
0,2004,4880.707047,6,3.0,0.90,0,1,1
1,1991,2372.074001,4,2.5,1.02,2,0,1
2,1999,2961.434069,2,2.0,1.02,0,1,1
3,1988,2803.218334,5,1.5,1.02,1,1,1
4,1987,2530.875697,4,2.0,0.87,3,0,1


In [9]:
y_train.head()

0    7.75606
1    3.38611
2    4.09515
3    3.68282
4    3.47681
Name: PRICE, dtype: float64

Rearrange dataframe for SageMaker training and scale price.

In [10]:
train_df = pd.DataFrame(data=x_train_np)
train_df.columns = x_train.columns
train_df["PRICE"] = y_train / 100000
first_col = train_df.pop("PRICE")
train_df.insert(0, "PRICE", first_col)

test_df = pd.DataFrame(data=x_test_np)
test_df.columns = x_test.columns
test_df["PRICE"] = y_test.reset_index(drop=True) / 100000
first_col = test_df.pop("PRICE")
test_df.insert(0, "PRICE", first_col)

Save as both CSV and Numpy data types to demonstrate data type flexibility in model training.

In [11]:
# Save as CSV
train_df.to_csv(f"{train_dir}/train.csv", header=False, index=False)
test_df.to_csv(f"{test_dir}/test.csv", header=False, index=False)

# Save as Numpy
np.save(os.path.join(train_dir, "x_train.npy"), x_train_np)
np.save(os.path.join(test_dir, "x_test.npy"), x_test_np)
np.save(os.path.join(train_dir, "y_train.npy"), y_train)
np.save(os.path.join(test_dir, "y_test.npy"), y_test)

Upload the data to S3

In [12]:
s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(csv_s3_prefix, "train.csv")).upload_file(
    "data/train/train.csv"
)
s3_resource_bucket.Object(os.path.join(csv_s3_prefix, "test.csv")).upload_file("data/test/test.csv")
s3_resource_bucket.Object(os.path.join(numpy_train_s3_prefix, "x_train.npy")).upload_file(
    "data/train/x_train.npy"
)
s3_resource_bucket.Object(os.path.join(numpy_train_s3_prefix, "y_train.npy")).upload_file(
    "data/train/y_train.npy"
)
s3_resource_bucket.Object(os.path.join(numpy_test_s3_prefix, "x_test.npy")).upload_file(
    "data/test/x_test.npy"
)
s3_resource_bucket.Object(os.path.join(numpy_test_s3_prefix, "y_test.npy")).upload_file(
    "data/test/y_test.npy"
)

### Scikit-learn

The first "level" of script mode is the ability to define your own training job, model, and inference process without any dependencies. This is done using a customized python script and pointing that script as the "entry point" when defining your SageMaker training estimator. There is no "out-of-the-box" random forest algorithm on SageMaker, but there is support for scikit-learn containers which does have random forest implementations, including regressors and classifiers. Here, we demonstrate the implementation of a custom random forest regressor to predict housing prices using our synthetic housing data set.

Script Mode in SageMaker allows you to take control of the training and inference process without having to go through the trouble of creating and maintaining your own docker containers. For example, if you want to use a scikit-learn algorithm, just use the AWS-provided scikit-learn container and pass it your own training and inference code. On your behalf, the SageMaker Python SDK will package this entry point script (which can be your training and/or inference code), upload it to S3, and set two environment variables that are read at runtime and load the custom training and inference functions from the entry point script. These two environment variables are `SAGEMAKER_SUBMIT_DIRECTORY` which is set to the S3 path of the package and `SAGEMAKER_PROGRAM` which is set to the name of the script (which in our case is `train_deploy_scikitlearn_without_dependencies.py`).

The process is the same if you want to use an XGBoost model (use the XGBoost container) or a custom PyTorch model (use the PyTorch container). Since you're passing in your own script (which is why we call it "script mode"), you get to define the model, the training process, and the inference process as well.

Below we include an entry point script called `train_deploy_scikitlearn_without_dependencies.py` which contains our custom training and inference code. 

In [14]:
pip install -U sagemaker sklearn

  Using cached sagemaker-2.129.0-py2.py3-none-any.whl
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.24.1
    Uninstalling sagemaker-2.24.1:
      Successfully uninstalled sagemaker-2.24.1
Note: you may need to restart the kernel to use updated packages.


In [15]:
hyperparameters = {"max_depth": 20, "n_jobs": 4, "n_estimators": 120}

if enable_local_mode_training:
    train_instance_type = "local"
    inputs = {"train": f"file://{train_dir}", "test": f"file://{test_dir}"}
else:
    train_instance_type = "ml.c5.xlarge"
    inputs = {"train": csv_train_s3_uri, "test": csv_test_s3_uri}

estimator_parameters = {
    "entry_point": "train_deploy_scikitlearn_without_dependencies.py",
    "source_dir": "scikitlearn_script",
    "framework_version": "1.0-1",
    "py_version": "py3",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "randomforestregressor-model",
}

estimator = SKLearn(**estimator_parameters)
estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: randomforestregressor-model-2023-01-25-16-28-11-807


2023-01-25 16:28:12 Starting - Starting the training job...
2023-01-25 16:28:29 Starting - Preparing the instances for training......
2023-01-25 16:29:23 Downloading - Downloading input data...
2023-01-25 16:30:08 Training - Downloading the training image..2023-01-25 16:30:18,063 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-01-25 16:30:18,066 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-25 16:30:18,072 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-01-25 16:30:18,296 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-25 16:30:18,305 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-25 16:30:18,314 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-25 16:30:18,321 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "ad

After the estimator finishes training, we can deploy it to a SageMaker endpoint.

In [16]:
existing_endpoints = sess.sagemaker_client.list_endpoints(
    NameContains=sklearn_endpoint_name, MaxResults=30
)["Endpoints"]
if not existing_endpoints:
    sklearn_predictor = estimator.deploy(
        initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=sklearn_endpoint_name
    )
else:
    sklearn_predictor = Predictor(
        endpoint_name="randomforestregressor-endpoint",
        sagemaker_session=sess,
        serializer=NumpySerializer(),
        deserializer=NumpyDeserializer(),
    )

INFO:sagemaker:Creating model with name: randomforestregressor-model-2023-01-25-16-30-55-845
INFO:sagemaker:Creating endpoint-config with name randomforestregressor-endpoint
INFO:sagemaker:Creating endpoint with name randomforestregressor-endpoint


----------!

Then we can use the SageMaker endpoint to make predictions.

In [26]:
x_test.values[0]

array([2.00100000e+03, 2.95349534e+03, 4.00000000e+00, 1.50000000e+00,
       9.80000000e-01, 1.00000000e+00, 0.00000000e+00, 1.00000000e+00])

In [31]:
sklearn_predictor.predict(x_test)

array([8.18585150e-05, 8.28684425e-05, 8.09093251e-05, 8.16602900e-05,
       8.16602900e-05, 8.24719075e-05, 8.19989508e-05, 8.31089075e-05,
       8.23830242e-05, 8.17438317e-05, 8.17412975e-05, 8.21355925e-05,
       8.06022810e-05, 8.16628242e-05, 8.21673700e-05, 8.17412975e-05,
       8.12814143e-05, 8.10113543e-05, 8.09092410e-05, 8.09223660e-05,
       8.21355925e-05, 8.08191043e-05, 8.17039658e-05, 8.10439260e-05,
       8.15642025e-05, 8.11468135e-05, 8.10939885e-05, 8.09456876e-05,
       8.24719075e-05, 8.05686235e-05, 8.25255008e-05, 8.20049542e-05,
       8.17438317e-05, 8.05711576e-05, 8.17565658e-05, 8.09223660e-05,
       8.09067068e-05, 8.13087768e-05, 8.18585150e-05, 8.25255008e-05,
       8.25255008e-05, 8.08137058e-05, 8.20024200e-05, 8.23830242e-05,
       8.23830242e-05, 8.17412975e-05, 8.33042792e-05, 8.11910518e-05,
       8.08762851e-05, 8.11910518e-05, 8.23301292e-05, 8.17529493e-05,
       8.20084018e-05, 8.15057408e-05, 8.21869358e-05, 8.27185217e-05,
      

### PyTorch

The second "level" of script mode is the ability to modularize and logically organize your custom training jobs, models, and inference processes.

Sometimes keeping all your code in one Python file can be unwieldy. Script Mode gives you the flexibility to parse out your code into multiple Python files. To illustrate this feature we build a custom PyTorch model and logically separate the model definition from the the training and inference logic. This is done by stipulating the source directory when defining your SageMaker training estimator (illustrated below). Once again, the model is not supported "out-of-the-box", but the PyTorch framework is and can be leveraged in the same manner as scikit-learn was in the previous example.

In this PyTorch example, we want to separate the actual neural network definition from the rest of the code by putting it into its own file as demonstrated in the `pytorch_script/` folder.

In [18]:
hyperparameters = {"epochs": 25, "batch_size": 128, "learning_rate": 0.01}

if enable_local_mode_training:
    train_instance_type = "local"
    inputs = {"train": f"file://{train_dir}", "test": f"file://{test_dir}"}
else:
    train_instance_type = "ml.c5.xlarge"
    inputs = {"train": numpy_train_s3_uri, "test": numpy_test_s3_uri}

estimator_parameters = {
    "entry_point": "train_deploy_pytorch_without_dependencies.py",
    "source_dir": "pytorch_script",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "pytorch-model",
    "framework_version": "1.5",
    "py_version": "py3",
}

estimator = PyTorch(**estimator_parameters)
estimator.fit(inputs)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-model-2023-01-25-16-36-26-892


2023-01-25 16:36:27 Starting - Starting the training job...
2023-01-25 16:36:56 Starting - Preparing the instances for training......
2023-01-25 16:37:48 Downloading - Downloading input data...
2023-01-25 16:38:18 Training - Downloading the training image...
2023-01-25 16:38:49 Uploading - Uploading generated training modelbash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-01-25 16:38:43,426 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-01-25 16:38:43,429 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-25 16:38:43,438 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-01-25 16:38:43,440 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-01-25 16:38:43,628 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-01-25 16:38:43,

Again, after the estimator finishes training, we can deploy it to a SageMaker endpoint.

In [19]:
existing_endpoints = sess.sagemaker_client.list_endpoints(
    NameContains=pytorch_endpoint_name, MaxResults=30
)["Endpoints"]
if not existing_endpoints:
    pytorch_predictor = estimator.deploy(
        initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=pytorch_endpoint_name
    )
else:
    pytorch_predictor = Predictor(
        endpoint_name="pytorch-endpoint",
        sagemaker_session=sess,
        serializer=JSONSerializer(),
        deserializer=JSONDeserializer(),
    )

INFO:sagemaker:Creating model with name: pytorch-model-2023-01-25-16-39-42-860
INFO:sagemaker:Creating endpoint-config with name pytorch-endpoint
INFO:sagemaker:Creating endpoint with name pytorch-endpoint


-----!

Then we can use the endpoint to make predictions.

In [20]:
pytorch_predictor.serializer = JSONSerializer()
pytorch_predictor.deserializer = JSONDeserializer()

pytorch_predictor.predict(x_test.values[0])

5.702870845794678

In [26]:
x_test.values[0]

array([2.00100000e+03, 2.95349534e+03, 4.00000000e+00, 1.50000000e+00,
       9.80000000e-01, 1.00000000e+00, 0.00000000e+00, 1.00000000e+00])

### XGBoost

The third "level" of script mode is the ability to bring your own libraries and dependencies to support custom functionality within your models, training jobs, and inference processes. This supercharges your customization options, and allows you to import libraries you have created yourself or Python packages hosted on PyPi.

Perhaps the number of Python files you have is becoming unwieldy now or you want more organization. In this scenario, you might be tempted to create your own Python library. Or maybe you wish to implement a function not currently supported by SageMaker in the training phase (such as  k-fold cross validation).

Script Mode supports adding custom libraries and those libraries don't have to be in the same directory as your entry point Python script. You simply need to stipulate the custom library or other dependencies when defining your SageMaker training estimator (illustrated below). SageMaker will copy the library folder to the same folder where the entry point script is located when the training job is kicked off.

In this example, we implement k-fold cross validation for an XGBoost model using a custom built library called `my_custom_library`. While XGBoost is supported "out-of-the-box" on SageMaker, that version does not support k-fold cross validation for training. Thus we use script mode to leverage the supported XGBoost container and the concomitant flexibility to include our custom libraries and dependencies. 


In [21]:
hyperparameters = {"num_round": 6, "K": 5}

if enable_local_mode_training:
    train_instance_type = "local"
    inputs = {"train": f"file://{train_dir}"}
else:
    train_instance_type = "ml.c5.xlarge"
    inputs = {"train": csv_s3_uri}

estimator_parameters = {
    "entry_point": "train_deploy_xgboost_with_dependencies.py",
    "source_dir": "xgboost_script",
    "dependencies": ["my_custom_library"],
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "xgboost-model",
    "framework_version": "1.0-1",
    "py_version": "py3",
}

estimator = XGBoost(**estimator_parameters)
estimator.fit(inputs)

INFO:sagemaker:Creating training-job with name: xgboost-model-2023-01-25-16-42-15-727


2023-01-25 16:42:16 Starting - Starting the training job...
2023-01-25 16:42:32 Starting - Preparing the instances for training......
2023-01-25 16:43:24 Downloading - Downloading input data.....[2023-01-25 16:44:26.551 ip-10-0-233-202.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Invoking user training script.
INFO:sagemaker-containers:Module train_deploy_xgboost_with_dependencies does not provide a setup.py. 
Generating setup.py
INFO:sagemaker-containers:Generating setup.cfg
INFO:sagemaker-containers:Generating MANIFEST.in
INFO:sagemaker-containers:Installing module with the following command:
/miniconda3/bin/python3 -m pip install . 
Processing /opt/ml/code
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created

After we train the model with k-fold cross validation, we can deploy it to a SageMaker endpoint.

In [22]:
existing_endpoints = sess.sagemaker_client.list_endpoints(
    NameContains=xgboost_endpoint_name, MaxResults=30
)["Endpoints"]

In [23]:
existing_endpoints = sess.sagemaker_client.list_endpoints(
    NameContains=xgboost_endpoint_name, MaxResults=30
)["Endpoints"]
if not existing_endpoints:
    xgboost_predictor = estimator.deploy(
        initial_instance_count=1, instance_type="ml.m5.xlarge", endpoint_name=xgboost_endpoint_name
    )
else:
    xgboost_predictor = Predictor(
        endpoint_name="xgboost-endpoint",
        sagemaker_session=sess,
        serializer=CSVSerializer(),
        deserializer=JSONDeserializer(),
    )

INFO:sagemaker:Creating model with name: xgboost-model-2023-01-25-16-44-59-131
INFO:sagemaker:Creating endpoint-config with name xgboost-endpoint
INFO:sagemaker:Creating endpoint with name xgboost-endpoint


-----!

Then you can use the endpoint to make predictions.

In [24]:
xgboost_predictor.serializer = CSVSerializer()
xgboost_predictor.deserializer = JSONDeserializer()
xgboost_predictor.predict(x_test.values[0])[0]

[0.053864985704422]

In [26]:
x_test.values[0]

array([2.00100000e+03, 2.95349534e+03, 4.00000000e+00, 1.50000000e+00,
       9.80000000e-01, 1.00000000e+00, 0.00000000e+00, 1.00000000e+00])

### Cleanup

In [ ]:
resources = (
    [sklearn_endpoint_name, sklearn_predictor],
    [pytorch_endpoint_name, pytorch_predictor],
    [xgboost_endpoint_name, xgboost_predictor],
)

for resource in resources:
    existing_endpoints = sess.sagemaker_client.list_endpoints(
        NameContains=resource[0], MaxResults=30
    )["Endpoints"]
    if existing_endpoints:
        resource[1].delete_endpoint(delete_endpoint_config=True)